## Notebook de l'exploration et pre-traitement des questions du corpus v2

### A. Pre-requis

In [1]:
# Importation des librairies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from bs4 import BeautifulSoup

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D

from sklearn.model_selection import train_test_split

In [2]:
# Chargement du dataset

dataset = pd.read_csv('dataset/stack_overflow_query_result.csv')

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Title          50000 non-null  object 
 1   Body           50000 non-null  object 
 2   Tags           50000 non-null  object 
 3   Id             50000 non-null  int64  
 4   Score          50000 non-null  int64  
 5   ViewCount      50000 non-null  int64  
 6   FavoriteCount  0 non-null      float64
 7   AnswerCount    50000 non-null  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 3.1+ MB


In [4]:
dataset.describe()

,Id,Score,ViewCount,FavoriteCount,AnswerCount
count,5.000000e+04,50000.000000,5.000000e+04,0.0,50000.00000
mean,1.562872e+06,25.357040,2.371003e+04,NaN,3.58756
std,7.694544e+05,217.521498,1.506203e+05,NaN,3.99613
min,4.000000e+00,-6.000000,2.900000e+01,NaN,0.00000
25%,9.314418e+05,1.000000,1.038000e+03,NaN,1.00000
50%,1.614784e+06,2.000000,2.637000e+03,NaN,3.00000
75%,2.239268e+06,7.000000,8.269000e+03,NaN,4.00000
max,2.776234e+06,20372.000000,1.192372e+07,NaN,131.00000


### B. Nettoyage

#### Formatage de la variable 'Tags'

In [5]:
# Nettoyer le formatage de la variables 'Tags' en enlevant '<' puis en supprimant le dernier caractere '>' et en decoupant la chaine par '>'

def process_tags(tags):
    """
    Transforme une chaîne de caractères contenant des tags délimités par '<' et '>'.
    
    Cette fonction remplace les caractères '<' dans la chaîne par des chaînes vides, 
    enlève le dernier caractère de la chaîne résultante, puis divise la chaîne restante 
    en une liste de sous-chaînes en utilisant '>' comme délimiteur.

    Args:
        tags (str): Une chaîne de caractères contenant des tags délimités par '<' et '>'.

    Returns:
        list: Une liste de chaînes, où chaque chaîne est un tag extrait de l'entrée.
    """
    return tags.replace('<', '')[0:-1].split('>')

dataset['Tags'] = dataset['Tags'].apply(process_tags)

#### Suppression de la variable 'FavoriteCount'

In [6]:
# Suppression de la variable 'FavoriteCount' du dataset.
# Cette variable est vide du fait d'un bug avec Stack Exchange Data Explorer

dataset.drop(columns='FavoriteCount', inplace=True)

#### Tokenisation de la variable 'Title'

In [7]:
# Fonction de tokenisation de la variable 'Title'

def gen_token(word):
    """
    Cette fonction prend en entrée une chaîne de caractères (word) et retourne
    une liste de tokens générés à partir de cette chaîne en utilisant un 
    tokenizer basé sur des expressions régulières. Elle conserve également
    certains caractères spéciaux comme le '#' et l'apostrophe.

    Args:
    word (str): La chaîne de caractères à tokenizer.

    Returns:
    list: Une liste contenant les tokens générés à partir de la chaîne de caractères.
    """
    tokenizer = nltk.RegexpTokenizer(r"[\w#'.]+")
    return tokenizer.tokenize(word)

In [8]:

# On applique la fonction a la variable 'Title'
dataset['title_tokenize'] = dataset['Title'].apply(gen_token)

#### Tokenisation de la variable 'Body'

In [9]:
# Fonction de tokenisation de la variable 'Body'

dataset['body_tokenize'] = dataset['Body'].apply(gen_token)

### C. Pre-processing

In [10]:
# Importe le corpus des stopwords pour l'anglais depuis NLTK
stopwords = nltk.corpus.stopwords.words('english')

# Crée un ensemble de mots à partir du corpus de mots anglais disponible dans NLTK
words = set(nltk.corpus.words.words())

# Initialise un objet lemmatizer de la classe WordNetLemmatizer pour le traitement de lemmatisation
lemmatizer = WordNetLemmatizer()

In [11]:
def preproc_text(text):
    """
    Cette fonction prend en entrée un tweet (chaîne de caractères) et retourne
    une version prétraitée du tweet après avoir effectué la tokenisation, la
    suppression des stopwords, la lemmatisation et le filtrage des tokens ayant moins de 3 caractères.

    Args:
    text (str): Le tweet à prétraiter.

    Returns:
    str: Le tweet prétraité sous forme de chaîne de caractères, avec les tokens
         filtrés, séparés par des espaces.
    """

    # Enlever les balises HTML
    soup = BeautifulSoup(text, "lxml").get_text()

    # Tokenisation modifiée pour inclure des caractères comme "#" et "."
    tokenizer = nltk.RegexpTokenizer(r"[\w#'.]+").tokenize(soup)

    # Supprimer les stopwords
    words_w_stopwords = [i for i in tokenizer if i not in stopwords]

    # Lemmatisation
    words_lemmatize = (lemmatizer.lemmatize(w) for w in words_w_stopwords)

    # Supprimer les tokens de moins de 3 caractères mais conserver ceux avec des caractères spéciaux
    words_w_stopwords_and_3char = [w.lower() for w in words_lemmatize if len(w) >= 2]

    return ' '.join(words_w_stopwords_and_3char)

In [12]:
# Pre-processing de la variable 'Body'

dataset['preproc_body'] = dataset['Body'].apply(preproc_text)

In [13]:
# Pre-processing de la variable 'Title'

dataset['preproc_title'] = dataset['Title'].apply(preproc_text)

/var/folders/b8/t710d8ys4x55cg_q12zdpnf00000gn/T/ipykernel_21499/2928602134.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml").get_text()


In [59]:
from gensim.models import Word2Vec

sentences = [dataset['preproc_title']]

model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [62]:
word = 'gettimeofday'
if word in model.wv:
    print(f"Vecteur pour le mot '{word}': {model.wv[word]}")
else:
    print(f"Le mot '{word}' n'est pas présent dans le vocabulaire.")

Le mot 'gettimeofday' n'est pas présent dans le vocabulaire.
